In [1]:
import warnings
import numpy as np 
import pandas as pd
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 12

In [2]:
test_df = pd.read_csv('input/test_df.csv', index_col='img')
test_df

,1,2,3,4,defects
img,,,,,
008ef3d74.jpg,356336 4 356587 11 356838 18 357089 25 357340 ...,375439 5 375687 14 375935 24 376182 34 376430 ...,NaN,NaN,2
c98404d89.jpg,NaN,113365 14 113595 40 113824 67 114053 94 114283...,NaN,NaN,1
cdc267505.jpg,NaN,108278 11 108513 32 108749 52 108984 73 109219...,NaN,NaN,1
2012a6d02.jpg,NaN,217692 17 217948 51 218204 85 218459 103 21871...,NaN,NaN,1
00c88fed0.jpg,10474 7 10728 15 10983 18 11239 21 11494 24 11...,13428 8 13684 24 13940 39 14196 55 14452 71 14...,NaN,NaN,2
...,...,...,...,...,...
2279bbcc4.jpg,NaN,NaN,NaN,NaN,0
394c73130.jpg,NaN,NaN,NaN,NaN,0
3505b6c74.jpg,NaN,NaN,NaN,NaN,0


In [3]:
def load_df(path):
    df = pd.read_csv(path)
    df['ClassId'] = df['ClassId'].astype(int)
    df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
    df = df.reindex(columns=[1, 2, 3, 4])
    df['defects'] = df.count(axis=1)
    df = df.rename({'ImageId': 'img'}, axis='columns')
    return df

In [4]:
unet18 = load_df('output/predictions/Unet(ResNet18).csv')
unet18

ClassId,1,2,3,4,defects
ImageId,,,,,
000a4bcdd.jpg,39765 7 40018 34 40272 38 40528 39 40784 39 41...,40558 2 41437 13 41690 19 41943 24 42198 26 42...,NaN,NaN,2
002af848d.jpg,NaN,NaN,130686 39 130933 53 131180 64 131429 73 131602...,291824 12 292078 15 292333 17 292587 19 292842...,2
005f02e20.jpg,NaN,NaN,322248 3 322501 8 322750 15 323001 21 323254 2...,NaN,1
008ef3d74.jpg,360149 23 360400 36 360652 47 360906 50 361159...,360152 8 360401 32 360653 41 360906 48 361160 ...,NaN,NaN,2
0095cd374.jpg,NaN,NaN,18488 24 18740 30 18995 33 19249 36 19292 2 19...,NaN,1
...,...,...,...,...,...
ff1020719.jpg,NaN,NaN,145863 46 145910 6 146117 56 146372 57 146628 ...,NaN,1
ff52f9fe2.jpg,1379 3 1630 12 1884 15 2138 18 2393 19 2648 20...,1632 8 1885 12 2140 14 2395 16 2650 17 2906 16...,56611 19 56864 23 57119 25 57374 26 57628 28 5...,NaN,3
ff5483763.jpg,NaN,NaN,168744 31 168992 66 169238 79 169491 85 169744...,NaN,1


In [5]:
def get_mask(df, image_id):
    labels = df.loc[image_id][:4]
    masks = np.zeros((256, 1600, 4), dtype=np.uint8)
    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            label = label.split(' ')
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            mask = np.zeros(256 * 1600, dtype=np.uint8)
            for pos, le in zip(positions, length):
                mask[pos:(pos + le)] = 1
            masks[:, :, idx] = mask.reshape(256, 1600, order='F')
    return masks

def show_mask(masks, path, image_id):
    palet = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249, 50, 12)]
    img = cv2.imread(os.path.join(path, image_id))
    fig, ax = plt.subplots(figsize=(15, 15))
    title = image_id + ' '
    for ch in range(4):
        contours, _ = cv2.findContours(masks[:, :, ch], cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        for i in range(0, len(contours)):
            cv2.polylines(img, contours[i], True, palet[ch], 2)
            title += str(ch + 1)
    ax.set_title(title)
    ax.imshow(img)
    plt.show()

In [6]:
def metrics(pred, truth):
    pred = np.transpose(pred, (2, 0, 1))  # 4x256x1600
    pred = pred.reshape(4,-1)
    truth = np.transpose(truth, (2, 0, 1))  # 4x256x1600
    truth = truth.reshape(4,-1)
    assert (pred.shape == truth.shape)

    p = (pred > 0.5)
    t = (truth > 0.5)
    
    neg_index = np.nonzero(t.sum(-1) == 0)[0]
    pos_index = np.nonzero(t.sum(-1) >= 1)[0]

    neg = (p.astype(float).sum(-1) == 0).astype(float)
    dice_pos = 2 * (p & t).sum(-1) / ((p.astype(float) + t.astype(float)).sum(-1))
    iou_pos = (p & t).sum(-1).astype(float) / ((p | t).sum(-1))

    dice = np.zeros(4)
    dice[neg_index] = neg[neg_index]
    dice[pos_index] = dice_pos[pos_index]
    
    iou = np.zeros(4)
    iou[neg_index] = neg[neg_index]
    iou[pos_index] = iou_pos[pos_index]
    
    classes = []
    for i in range(4):
        if dice[i] > 0.2:
            if i in neg_index:
                classes.append('TN')
            else:
                classes.append('TP')
        else:
            if i in neg_index:
                classes.append('FP')
            else:
                classes.append('FN')    
    return dice, iou, classes

In [7]:
def show_compare(img, test, pred):
    test_mask = get_mask(test, img)
    show_mask(test_mask, 'input/train_images', img)
    if img in pred.index:
        pred_mask = get_mask(pred, img)
        show_mask(pred_mask, 'input/train_images', img)
        dice, iou, classes = metrics(pred_mask, test_mask)        
    else:
        classes = ['TN', 'TN', 'TN', 'TN']
        dice = test.loc[img][:4].notna().to_numpy().astype(float)
        iou = dice
    print('dice: {}\n IoU: {}\n classes: {}'.format(dice, iou, classes))        

In [8]:
def compare(test, pred):
    metr = []
    dices = []
    for img in test.index:
        test_mask = get_mask(test, img)
        if img in pred.index:
            pred_mask = get_mask(pred, img)
            dice, iou, classes = metrics(pred_mask, test_mask)        
        else:
            classes = ['TN', 'TN', 'TN', 'TN']
            dice = test.loc[img][:4].notna().to_numpy().astype(float)
        metr.append(classes)
        dices.append(dice)
    metr = np.array(metr)
    dices = np.array(dices)
    return pd.DataFrame(data=metr, index=test.index, columns=[1, 2, 3, 4]), pd.DataFrame(data=dices, index=test.index, columns=[1, 2, 3, 4])

In [ ]:
imgs = np.random.choice(test_df.index, 3, replace=False)
for img in imgs:
    show_compare(img, test_df, unet18)

In [9]:
warnings.filterwarnings('ignore')
classes18, dices18 = compare(test_df, unet18)

In [10]:
def class_metrics(df):
    metr = []
    for i in range(1, 5):
        TP = df[i].value_counts()['TP']
        TN = df[i].value_counts()['TN']
        FP = df[i].value_counts()['FP']
        FN = df[i].value_counts()['FN']        
        metr.append({'accuracy' : (TP + TN) / (TP + TN + FP + FN),
         'precision' : TP / (TP + FP),
         'recall': TP / (TP + FN)})
        print('class {} metr: {}'.format(i, metr[i-1]))
    return metr

In [14]:
dices18.describe()

,1,2,3,4
count,2515.000000,2515.000000,2515.000000,2515.000000
mean,0.441340,0.436849,0.341875,0.491146
std,0.484729,0.492482,0.401866,0.488858
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.488525
75%,1.000000,1.000000,0.747098,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [12]:
metr18 = class_metrics(classes18)

class 1 metr: {'accuracy': 0.8982107355864811, 'precision': 0.36363636363636365, 'recall': 0.7607361963190185}
class 2 metr: {'accuracy': 0.88389662027833, 'precision': 0.12538226299694188, 'recall': 0.8723404255319149}
class 3 metr: {'accuracy': 0.8532803180914513, 'precision': 0.8061440677966102, 'recall': 0.8035902851108765}
class 4 metr: {'accuracy': 0.9502982107355865, 'precision': 0.5637065637065637, 'recall': 0.9240506329113924}
